In [16]:
import requests
import time
import csv
from bs4 import BeautifulSoup
import re
import openpyxl as op
#from pyExcelerator import *   
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36 Edg/96.0.1054.53'} 
Contest=['playoffs','regular_seasons']
if __name__=="__main__":
    pages,pages2=990//150,175//150
    url_header='http://www.stat-nba.com/query.php?page='
    url_tail='&QueryType=game&GameType=season&Player_id=1717&order=1&crtcol=pts&PageNum=200'
    url_tail2='&QueryType=game&GameType=playoff&Player_id=1717&order=order0&crtcol=cseason&PageNum=200'  
    url_lists=[]
    url_lists2=[]
    for i in range(pages+1):
        url_lists.append(url_header+str(i)+url_tail)
    for i in range(pages2+1):
        url_lists2.append(url_header+str(i)+url_tail2)
    dataseasons=get_nba_all_data(1,url_lists)
    dataplayoff=get_nba_all_data(0,url_lists2)
    #print(len(datasets))
    Data=[dataplayoff,dataseasons]
    for i in range(2):
        write_data_to_file(i,Data[i])
        print(Contest[i]+"保存成功！")

playoffs保存成功！
regular_seasons保存成功！


In [14]:
def get_nba_all_data(con,url_lists):
    #获取所有比赛数据
    datasets=[]
    response=requests.get(url_lists[0],headers=headers)
    soup=BeautifulSoup(response.content.decode('utf-8'),'html.parser')
    index=1
    for data in soup.find_all('th'):
        if index==8+con or index==9+con:
            datasets.append('总'+data.text)
        elif index==11+con or index==12+con:
            datasets.append('三分'+data.text)
        elif index==14+con or index==15+con:
            datasets.append('罚球'+data.text)
        else:
            datasets.append(data.text)
        index+=1
    for item in url_lists:
        datasets.extend(get_nba_single_data(item))
    #print(datasets)
    return datasets

In [3]:
def get_nba_single_data(url):
    #获取1个页面比赛数据
    #url = 'http://stat-nba.com/query_team.php?page=0&QueryType=game&order=1&crtcol=date_out&GameType=season&PageNum=3000&Season0=2016&Season1=2017#label_show_result'  
    # html = requests.get(url).text
    response=requests.get(url,headers=headers,timeout=10)
    soup=BeautifulSoup(response.content.decode('utf-8'),'html.parser')
    #print (soup)
    list_data=[]
    for data in soup.find_all('td'):
        list_data.append(data.text)
    return list_data

In [4]:
def write_data_to_file(index,datasets):
    Index=24+index
    file=open(Contest[index]+'.txt','w',encoding='utf-8')
    sign=1
    for i in range(len(datasets)-1):
        file.write(datasets[i]+'\t')
        sign+=1
        if sign%(Index)==1:
            file.write('\n')
    file.close()
    
    excel=op.Workbook()
    sheet=excel.active
    for i in range(1,len(datasets)+1):
        if i%(24+index)==0:
            sheet.cell(i//Index,Index,str(datasets[i-1]))
        else:
            sheet.cell((i+Index)//(Index),i%(Index),str(datasets[i-1]))
    excel.save(Contest[index]+".xlsx")    